I have computed all the required data, including CAR, Earning surprise and change in holdings between announcements. the tables and the description of the main variables is below.
<br>
Next,I need the sustainability rating to the data. BUT, how should I proceed from here.
<br>
The sustainability data is monthly, while earning data is on a quarterly basis. 
<br>
Some options are:
1. Take the average portfolio sustainability score in a quarter, and choose the highest 10% every quarter (to get the ESG funds).
2. Same as one but with an annual score.
3. any other ideas? 
<br>
<br>

Also there is the question of timing the sustainability score and the earning report. Should I use the sustainability score which is in the same quarter as the earning report (say score on earning report date of May 15th), or the score that the report is regarding (say score on quarter end of Q1 or March 31st).
<br>

(The simplified version Moura-Dantas used, now makes a lot of sense and solves many issues)
<br><br>

Tables
<br>
Fund_Sust - contains information about the sustainability rating as a time series. 
<br>
1.  portfolio_score_date, as a monthly date
2.  portfolio_score - portfolio sustainability score
3.  crsp_fundno - unique fund identifier
4.  crsp_portno - unique portfolio identifier will be used to merge the data
<br><br>

holding_data - contains all the data about earnings, return and changes in fund positions.
<br>

1. crsp_portno - unique portfolio identifier unique portfolio identifier will be used to merge the data.
2. e_surprise - earning surprise
3. change_nbr_shares_total_before_e_annc - change in the number of shares the fund holds before the earning surprise.
4. car - the [-1,1] day CAR around the earning date. 




In [1]:
import pandas as pd
import re
import numpy as np
import math
from datetime import datetime, timedelta
import os
import time
import glob

pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_colwidth', -1)
# Reduce decimal points to 2
pd.options.display.float_format = '{:,.4f}'.format
#set my desired working directory
os.chdir("/Users/ariel/Dropbox/PhD/Michael CSR/Analysis")
#load time series of fund sustanability
fund_sust=pd.read_csv('Jupyter/fund_sust.csv')
fund_sust=fund_sust.drop(columns=['Unnamed: 0'])
#load data file
holding_data=pd.read_csv('Jupyter/holding_data.csv') #save
holding_data=holding_data.drop(columns=['Unnamed: 0'])


### Mutual Fund Sustainability sample table

In [2]:
fund_sust

,Name,fund_cusip,Global Category,portfolio_score_date,portfolio_score,crsp_fundno,crsp_portno
0,Invesco Charter A,141310,US Equity Large Cap Blend,201209,51.4100,"2,899.0000","1,022,003.0000"
1,Invesco Charter A,141310,US Equity Large Cap Blend,201212,50.9000,"2,899.0000","1,022,003.0000"
2,Invesco Charter A,141310,US Equity Large Cap Blend,201303,51.0100,"2,899.0000","1,022,003.0000"
3,Invesco Charter A,141310,US Equity Large Cap Blend,201306,51.4400,"2,899.0000","1,022,003.0000"
4,Invesco Charter A,141310,US Equity Large Cap Blend,201309,49.9600,"2,899.0000","1,022,003.0000"
5,Invesco Charter A,141310,US Equity Large Cap Blend,201312,50.6100,"2,899.0000","1,022,003.0000"
6,Invesco Charter A,141310,US Equity Large Cap Blend,201403,53.7100,"2,899.0000","1,022,003.0000"
7,Invesco Charter A,141310,US Equity Large Cap Blend,201406,51.6900,"2,899.0000","1,022,003.0000"
8,Invesco Charter A,141310,US Equity Large Cap Blend,201409,51.2000,"2,899.0000","1,022,003.0000"
9,Invesco Charter A,141310,US Equity Large Cap Blend,201412,52.4900,"2,899.0000","1,022,003.0000"


### Fund holdings and other information - Sample data

In [3]:
holding_data


,crsp_portno,cusip,FPEDATS,rdq,atq,saleq,prccq,mcap,e_surprise,change_nbr_shares_total_before_e_annc,cret,car
0,1000045,00101J10,2013-03-31,2013-05-01,"9,710.0000",821.0000,48.9400,"10,763.9125",-0.0004,"-3,048.0000",-0.0273,-0.0267
1,1000045,00101J10,2013-06-30,2013-07-31,"9,655.0000",833.0000,39.8500,"8,471.5122",0.0023,"206,425.0000",-0.0343,-0.0495
2,1000045,00101J10,2013-09-30,2013-11-20,"9,913.0000",846.0000,40.6600,"8,497.1675",-0.0002,"621,075.0000",0.0136,0.0101
3,1000045,00101J10,2013-12-31,2014-01-30,"9,875.0000",839.0000,40.4700,"7,416.7345",-0.0015,"-123,419.0000",-0.2153,-0.2200
4,1000045,00101J10,2014-03-31,2014-04-30,"10,147.0000",837.0000,29.9500,"5,304.4744",0.0017,"-511,742.0000",0.0340,0.0244
5,1000045,00101J10,2014-06-30,2014-07-30,"10,133.0000",849.0000,34.9400,"6,078.1973",0.0023,"-1,437,930.0000",0.0507,0.0733
6,1000045,00282410,2012-03-31,2012-04-18,"62,415.8010","5,283.6850",61.2900,"96,433.1957",0.0007,"-90,630.0000",-0.0057,-0.0110
7,1000045,00289620,2013-10-31,2013-11-21,"2,852.3960","1,033.2930",37.4800,"2,863.2846",0.0003,"540,522.0000",-0.0293,-0.0408
8,1000045,00289620,2014-04-30,2014-05-29,"2,566.3930",822.4280,36.7600,"2,675.2090",0.0005,"383,493.0000",0.0368,0.0340
9,1000045,00289620,2014-07-31,2014-08-28,"2,563.1760",890.6050,39.3400,"2,807.4204",0.0018,"-29,060.0000",-0.0631,-0.0659
